In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import sys
import os
sys.path.append(os.path.abspath('../'))
from api import API
from data import Data
import pandas as pd

## Closest Matching UG Playlist to an ED playlist

In [5]:
d = Data()

In [23]:
diffs, ugPIDs = d.closestMatchingUGPlaylist("Are & Be")

In [25]:
ugPIDs[0]

213590

In [26]:
diffs[0]

1.466596422333839

In [27]:
names, artists = d.getUGPlaylist(213590)

In [28]:
names

['Tiny Little Robots',
 'Shoot to Thrill',
 'Immigrant Song',
 'Welcome to Paradise',
 'Hysteria',
 'Supermassive Black Hole',
 'Uprising',
 "The Kids Aren't Alright",
 'Bad Habit',
 'Coming for You',
 'Tom Sawyer',
 'Brianstorm',
 'N.I.B.',
 'Children of the Grave - Remastered Version',
 "I'm Shipping Up To Boston",
 'The State Of Massachusetts',
 'Paradise City',
 'Welcome To The Jungle',
 'For Whom The Bell Tolls - Remastered',
 'A Moment of Violence',
 'Everything Went Numb',
 'Icky Thump',
 'Ace of Spades',
 'Come Out And Play (Keep ‘Em Separated)',
 'Enter Sandman',
 'The Trooper - 1998 Remastered Version',
 'Hallowed Be Thy Name - 1998 Remastered Version',
 'Holy Wars...The Punishment Due - 2004 Digital Remaster',
 'Black Dragon',
 'Collapse (Post-Amerika)',
 'Woman']

In [29]:
artists

['Cage The Elephant',
 'AC/DC',
 'Led Zeppelin',
 'Green Day',
 'Muse',
 'Muse',
 'Muse',
 'The Offspring',
 'The Offspring',
 'The Offspring',
 'Rush',
 'Arctic Monkeys',
 'Black Sabbath',
 'Black Sabbath',
 'Dropkick Murphys',
 'Dropkick Murphys',
 "Guns N' Roses",
 "Guns N' Roses",
 'Metallica',
 'Streetlight Manifesto',
 'Streetlight Manifesto',
 'The White Stripes',
 'Motörhead',
 'The Offspring',
 'Metallica',
 'Iron Maiden',
 'Iron Maiden',
 'Megadeth',
 'The Vines',
 'Rise Against',
 'Wolfmother']

## Predicting playlist avg features from playlist names

In [13]:
features, names = d.getUGPlaylistFeaturesAndNames()

> c:\users\adam khazi\desktop\ucl-dissertation-code\data.py(529)getUGPlaylistFeaturesAndNames()
-> playlistNames = [p['_id']['playlist_name'] for p in playlistNames]
(Pdb) continue


In [15]:
names[:50]

['BANDA',
 'Sex',
 'Praise',
 'GB',
 'PARTAY',
 'Calm',
 'Baby Making Music',
 'Lake',
 'stay woke',
 'summer 2017',
 'mom!',
 'Yoga',
 'Inspire',
 'old stuff',
 'Throwback ',
 'K pop',
 'EDM 2015',
 'XOXO',
 'New Tunes',
 'Summer',
 'throwbackk',
 '80s',
 'CHILE',
 'gym',
 'Soul',
 'She Will Be Loved',
 'Cruzin',
 'reggaeton',
 'Marathon',
 'TURNUP ',
 'rnb_hiphop',
 'Summer 2017',
 'Ear Candy',
 'My stuff',
 'Reggae',
 'Sexy ',
 'div',
 'Electronic',
 'Werk',
 'relax',
 'Jazz',
 'jams',
 'Lit songs',
 "90's country",
 'Reggeton',
 'Those days',
 'Throwback playlist',
 '❤️',
 'Chill Out',
 'good music']

In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import Normalizer

In [20]:
namesPD = pd.DataFrame(names, columns=['text'])

In [26]:
scaler = Normalizer()
scaledFeatures = scaler.fit_transform(features)

In [28]:
scaledFeatures[0].shape

(12,)

In [22]:
namesPD['text'] = namesPD['text'].apply(lambda x: x.lower())
namesPD['text'] = namesPD['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [23]:
max_features = 200
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(namesPD['text'].values)
X = tokenizer.texts_to_sequences(namesPD['text'].values)
X = pad_sequences(X)

In [29]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(12, activation='linear'))
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 8, 128)            25600     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 8, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 12)                2364      
Total params: 282,764
Trainable params: 282,764
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
scaledFeatures.shape

(1000000, 12)

In [32]:
X.shape

(92944, 8)

In [30]:
Y = scaledFeatures
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

ValueError: Found input variables with inconsistent numbers of samples: [92944, 1000000]